### Exploratory Data Analysis (EDA)

#### Initial Data Shape and Info

In [4]:
import pandas as pd
import numpy as np

# Load the NEWLY generated dataset (V3)
df = pd.read_csv('safaricom_ethiopia_dfs_regional_assurance_data_V3.csv')

print("Initial Data Shape and Info")
print(f"Shape: {df.shape}")
print(df.info())

print("\nSummary Statistics for Amount_ETB")
print(df['Amount_ETB'].describe())

print("\n Distribution of Target Variable (Is_Fraud) ")
# Check the level of class imbalance
print(df['Is_Fraud'].value_counts(normalize=True).mul(100).round(3).rename("Percentage").to_frame())

Initial Data Shape and Info
Shape: (100000, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Transaction_ID         100000 non-null  object 
 1   Transaction_Time       100000 non-null  object 
 2   Transaction_Type       100000 non-null  object 
 3   Sender_ID              100000 non-null  object 
 4   Amount_ETB             100000 non-null  float64
 5   System_Fee_ETB         100000 non-null  float64
 6   Sender_Region          100000 non-null  object 
 7   Receiver_Region        100000 non-null  object 
 8   Is_Cross_Region        100000 non-null  int64  
 9   Receiver_ID            100000 non-null  object 
 10  Billing_System_Status  100000 non-null  object 
 11  Is_Fraud               100000 non-null  int64  
 12  Fraud_Vector           803 non-null     object 
dtypes: float64(2), int64(2), object(9)
memory 

#### Basic Assurance & Regional Insights

In [7]:
print("\nA. Fraud/Leakage Rate by Region (Assurance Check) ")
regional_fraud_rates = df.groupby('Sender_Region')['Is_Fraud'].mean().mul(100).round(2).sort_values(ascending=False)
print(regional_fraud_rates.rename("Fraud Rate (%)"))

print("\n B. Fraud/Leakage Rate by Billing Status (Leakage Check) ")
billing_status_fraud_rates = df.groupby('Billing_System_Status')['Is_Fraud'].mean().mul(100).round(2).sort_values(ascending=False)
print(billing_status_fraud_rates.rename("Fraud Rate (%)"))

print("\n C. Transaction Type Risk ")
txn_type_fraud_rates = df.groupby('Transaction_Type')['Is_Fraud'].mean().mul(100).round(2).sort_values(ascending=False)
print(txn_type_fraud_rates.rename("Fraud Rate (%)"))


A. Fraud/Leakage Rate by Region (Assurance Check) 
Sender_Region
Oromia            0.88
Addis Ababa       0.81
South Ethiopia    0.62
Name: Fraud Rate (%), dtype: float64

 B. Fraud/Leakage Rate by Billing Status (Leakage Check) 
Billing_System_Status
TIMEOUT         23.01
RATING_ERROR    21.04
SUCCESS          0.31
Name: Fraud Rate (%), dtype: float64

 C. Transaction Type Risk 
Transaction_Type
Agent_Cash_In       1.15
Agent_Cash_Out      1.08
P2P_Transfer        1.00
Airtime_Purchase    0.00
B2C_Payment         0.00
Bill_Payment        0.00
Name: Fraud Rate (%), dtype: float64


#### 2: Creating Behavioral/Velocity Features

In [18]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('safaricom_ethiopia_dfs_regional_assurance_data_V3.csv') 
df['Transaction_Time'] = pd.to_datetime(df['Transaction_Time'])
df = df.sort_values(by='Transaction_Time').reset_index(drop=True)

# --- 1. Feature: Time-Based Transaction Volume (Velocity) ---

def calculate_rolling_velocity(group):
    # Set Transaction_Time as the index for time-based operations
    group_indexed = group.set_index('Transaction_Time')
    # Create a series of 1s for counting, ensure float type
    counts_series = pd.Series(1.0, index=group_indexed.index)
    # Calculate rolling count with closed='left' to exclude current transaction
    counts = counts_series.rolling('60Min', closed='left').count()
    return counts

velocity_results = []

for _, group in df.groupby('Sender_ID'):
    counts_series = calculate_rolling_velocity(group)
    # Align the results with the original group indices
    group_indices = group.index
    velocity_results.append(pd.Series(counts_series.values, index=group_indices, dtype=np.float64))

# Concatenate all results
final_velocity = pd.concat(velocity_results).sort_index()

# Fill NaNs with 0.0, then convert to nullable integer type
df['Txn_Count_Sender_1H'] = final_velocity.reindex(df.index).fillna(0.0).astype('Int64')

print("Feature 1 (Txn_Count_Sender_1H) calculated and safely cast.")

# --- 2. Feature: Agent Transaction Concentration (Collusion Risk) ---
agent_cash_out_df = df[df['Transaction_Type'] == 'Agent_Cash_Out'].copy()
agent_cash_out_df['Agent_Cust_Pair'] = agent_cash_out_df['Sender_ID'].astype(str) + '_' + agent_cash_out_df['Receiver_ID'].astype(str)

# Initialize the result Series as float
agent_pair_counts = pd.Series(0.0, index=df.index, dtype=np.float64)

for pair, group in agent_cash_out_df.groupby('Agent_Cust_Pair'):
    group_indexed = group.set_index('Transaction_Time').sort_index()
    
    # Create a series of 1s for counting
    counts_series = pd.Series(1.0, index=group_indexed.index)
    
    # Calculate the rolling count and ensure it is float64
    counts = counts_series.rolling('7D', closed='left').count()
    
    # Assign results back to the corresponding indices in the main result Series
    agent_pair_counts.loc[group.index] = counts.values.astype(np.float64)

# Fill NaNs with 0.0, then convert to nullable integer type
df['Agent_Cust_Pair_Count_7D'] = agent_pair_counts.fillna(0.0).astype('Int64')

# Drop the temporary pair key column
df.drop(columns=['Agent_Cust_Pair'], inplace=True, errors='ignore')

print("Feature 2 (Agent_Cust_Pair_Count_7D) calculated and safely cast.")

# --- 3. Feature: Behavioral (Night-Time Transaction Risk) ---
df['Is_Night_Time_Txn'] = df['Transaction_Time'].dt.hour.apply(lambda x: 1 if x >= 22 or x <= 5 else 0)

print("Feature 3 (Is_Night_Time_Txn) calculated.")

print("\n--- Features Created and Sample Output ---")
print(df[['Transaction_Time', 'Sender_ID', 'Transaction_Type', 'Amount_ETB', 'Txn_Count_Sender_1H', 'Agent_Cust_Pair_Count_7D', 'Is_Night_Time_Txn', 'Is_Fraud']].head(10))

# Save the enriched dataset for the next phase
df.to_csv('safaricom_ethiopia_dfs_enriched_data_V10_final.csv', index=False)
print("\n--- Enriched Dataset saved to safaricom_ethiopia_dfs_enriched_data_V10_final.csv ---")

Feature 1 (Txn_Count_Sender_1H) calculated and safely cast.
Feature 2 (Agent_Cust_Pair_Count_7D) calculated and safely cast.
Feature 3 (Is_Night_Time_Txn) calculated.

--- Features Created and Sample Output ---
     Transaction_Time   Sender_ID Transaction_Type  Amount_ETB  \
0 2025-01-01 00:00:00  CUST003417    Agent_Cash_In     7175.01   
1 2025-01-01 00:01:00  CUST003884    Agent_Cash_In    13717.43   
2 2025-01-01 00:01:00  CUST000380    Agent_Cash_In     2007.56   
3 2025-01-01 00:01:00  CUST010227     P2P_Transfer      167.74   
4 2025-01-01 00:01:00  CUST006125      B2C_Payment     3092.24   
5 2025-01-01 00:01:00  CUST010935   Agent_Cash_Out    10837.41   
6 2025-01-01 00:02:00  CUST004992     P2P_Transfer     4957.07   
7 2025-01-01 00:02:00  CUST003217   Agent_Cash_Out     7689.27   
8 2025-01-01 00:02:00  CUST010292     P2P_Transfer     4051.61   
9 2025-01-01 00:02:00  CUST010151     P2P_Transfer      160.62   

   Txn_Count_Sender_1H  Agent_Cust_Pair_Count_7D  Is_Night_Tim